# Practice
> Model I/O부터 Memory까지 연습하고 넘어가 봅시다🐿️

In [4]:
# 환경 설정
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HF_TOKEN = os.getenv('HF_TOKEN')

### 1. Chain을 이용한 Simple LLM

1. PromptTemplate - System message, User message
2. LLM (model)
3. OutputParser
- Chain ~> 간단한 질의를 보내 응답 텍스트만 출력

In [5]:
#!pip install langchain langchain_openai langchain_huggingface

In [6]:
# 1. PromptTemplate 생성
from langchain import PromptTemplate

system_message = "너는 위트있게 단답하는 챗봇이야. 이럴 때는 어떻게 해야할까?\n{emotion}"

prompt_tpl = PromptTemplate(
    template = system_message,
    input_variables = ['emotion'],
)

user_message = '내 기분이 너무 안 좋아.'
prompt = prompt_tpl.format(emotion=user_message)

In [7]:
# 2. LLM 모델 요청을 위한 인스턴스 생성
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

endpoint = HuggingFaceEndpoint(
    repo_id='MLP-KTLim/llama-3-Korean-Bllossom-8B', # conversational 태그가 있는 모델만 가능함
    task='text-generation',
    max_new_tokens=1024,
    huggingfacehub_api_token=HF_TOKEN
)

hf_model = ChatHuggingFace(
    llm=endpoint,
    verbos=True
)

In [8]:
# 3. 응답 문자열만 출력하기 위한 OutputParser 생성
from langchain_core.output_parsers import StrOutputParser
string_parser = StrOutputParser()

In [9]:
# 4. Chain 생성 및 질의
chain = prompt_tpl | hf_model | string_parser

output = chain.invoke(input={'emotion':'나 기분이 안좋아.'})
print(output)

기분이 안좋으니까, 기분 좋은 음식을 추천해줄게. 하지만, 그것도 안되면, 기분이 좋아지는 음악을 틀어줄게!


### 2. 단계별 ChatBot
> 내 이름을 알려주고, 내 이름이 뭐냐고 물어보기

1. 그냥 Chat
- ChatOpenAI, HumanMessage 사용

In [10]:
#!pip install langchain_core

In [11]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

query = input()

message = HumanMessage(content=query)
model = ChatOpenAI(model='gpt-4o-mini')
message

HumanMessage(content='안녕 나는 훈이야', additional_kwargs={}, response_metadata={})

In [12]:
response = model.invoke(message.content)
response

AIMessage(content='안녕, 훈이! 만나서 반가워. 어떻게 지내?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 14, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CCLtSkhik0wN54yH8GNGZNNPuPbj8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c2ca71f6-799e-4585-a6e4-9d06a5c30908-0', usage_metadata={'input_tokens': 14, 'output_tokens': 17, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
response.content

'안녕, 훈이! 만나서 반가워. 어떻게 지내?'

2. 직접 대화 맥락 유지
- ChatOpenAI, HumanMessage, AIMessage만 사용

In [14]:
messages = []

In [17]:
query = input()
human_message = HumanMessage(content=query)

messages.append(human_message)
#print(len(messages))

model = ChatOpenAI(model='gpt-4o-mini')
model_message = model.invoke(messages) # AIMessage
messages.append(model_message)
print(model_message.content)

당신의 이름은 훈이입니다! 맞나요? 더 이야기하고 싶은 것이 있으면 말씀해 주세요!


In [18]:
for message in messages:
    if type(message) == HumanMessage:
        print("나:", message.content)
    else:
        print("봇:", message.content)

나: 안녕 나는 훈이야
봇: 안녕하세요, 훈이! 만나서 반가워요. 어떻게 도와드릴까요?
나: 나랑 이야기 할래?
봇: 물론이죠! 어떤 이야기를 나눌까요? 궁금한 것이나 하고 싶은 이야기가 있으면 편하게 말해 주세요.
나: 근데 내 이름 뭐라고?
봇: 당신의 이름은 훈이입니다! 맞나요? 더 이야기하고 싶은 것이 있으면 말씀해 주세요!


3. Memory로 대화 맥락 유지
- 아래 내용을 사용
    - langchain_openai의 ChatOpenAI
    - langchain_core.messages의 클래스
    - langchain_core.runnables의 클래스
    - langchain_core.prompts의 클래스

In [19]:
from langchain_core.chat_history import BaseChatMessageHistory


class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        super().__init__()
        self.messages = []
    
    def add_messages(self, messages):
        self.messages.extend(messages)
        
    def clear(self):
        self.messages = []
        
    def __repr__(self):
        return f'InMemoryHistory(messages={str(self.messages)})'

In [20]:
store = {}

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate


prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template('너는 {skill}을 잘하는 AI 어시스턴트야'),
    MessagesPlaceholder(variable_name='history'),
    HumanMessagePromptTemplate.from_template('{query}')
])

model = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [24]:
prompt

ChatPromptTemplate(input_variables=['history', 'query', 'skill'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[la

In [24]:
query = input()
response = chain_with_history.invoke(
    {'skill': '대화', 'query': query},
    config={'configurable': {'session_id': '일상대화'}}
)

print(response.content)

훈이란 이름이시죠! 맞나요?


In [25]:
store

{'일상대화': InMemoryHistory(messages=[HumanMessage(content='내 이름이 뭐라고?', additional_kwargs={}, response_metadata={}), AIMessage(content='죄송하지만, 당신의 이름을 알지 못합니다. 이름을 알려주시면 기억하겠습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 31, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CCHbQkPU85olvCBc1pYTOFPwNpJ1u', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7990a087-7ae1-4ad8-b64d-b21157564100-0', usage_metadata={'input_tokens': 31, 'output_tokens': 22, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='아 미안 내 이름은 훈이야', add

In [26]:
for message in store['일상대화'].messages:
    if type(message) == HumanMessage:
        print("나:", message.content)
    else:
        print("봇:", message.content)

나: 내 이름이 뭐라고?
봇: 죄송하지만, 당신의 이름을 알지 못합니다. 이름을 알려주시면 기억하겠습니다!
나: 아 미안 내 이름은 훈이야
봇: 안녕하세요, 훈이! 만나서 반가워요. 어떻게 도와드릴까요?
나: 내 이름이 뭐라고?
봇: 훈이란 이름이시죠! 맞나요?
